In [2]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ファイル操作
import os
import glob

# スクレイピング
import requests
from bs4 import BeautifulSoup

In [33]:
url = "https://seesaawiki.jp/hololivetv/d/%c0%b1%b3%b9%a4%b9%a4%a4%a4%bb%a4%a4%a1%da%b2%ce%be%a7%b3%da%b6%ca%b0%ec%cd%f7%a1%db"

r = requests.get(url)

# soup = BeautifulSoup(r.content, "html.parser")
# 文字化けは以下のコードで直った（渡す前にdecodeして解決する方法だが、BS4の方で解決は難しいのか？）
soup = BeautifulSoup(r.content.decode("euc-jp", "ignore"), "html.parser")

# 取得できているか確認
print(soup.select("h1"))

[<h1><a href="https://seesaawiki.jp/hololivetv/">ホロライブ非公式wiki</a></h1>]


In [43]:
# 歌枠全体を管理しているテーブル
streams = soup.find_all("table", {"id":"content_block_13"})[0]
rows = streams.find_all("tr")

# セルごとの要素を抽出
# メンバー、公開日、タイトル、曲数、備考
# 現在取得できない動画には非公開と記載されているものが多い

# 枠そのものはダブルチェックくらいにしか使わないので一旦保留
# 他のデータをcsv形式で保存して使えるようにしておく

In [127]:
# 楽曲を管理しているテーブル
songs = soup.find_all("table", {"id":"content_block_19"})[0]
rows = songs.find_all("tr")

# リンクがある曲だけ取得する
def check_cell(cells):
    # リンクそのものが存在しない場合はfalse
    if cells[2].a == None:
        return False
    
    href = cells[2].a["href"]
    # 短縮系もあるため"youtu"で判別
    if "youtu" not in href:
        return False

    # 備考に非公開とある場合false
    text = cells[3].text
    if "非公開" in text:
        return False
    return True

# リンクから動画IDと開始時間を取得する
def get_videoid_time(cells):
    href = cells[2].a["href"]
    if "youtube" in href:
        stratid = href.find("v=")+2
        endid = href.find("&t=")
        videoid = href[stratid:endid]
        time = href[endid+3:-1]
    if "youtu.be" in  href:
        stratid = href.find(".be")+4
        endid = href.find("?t=")
        videoid = href[stratid:endid]
        time = href[endid+3:]
    return videoid, time
    
# メンバーと備考からアーティストを取得する
def get_artist(cells):
    artists = []
    member = list(cells[0].children)
    for i in range(len(member)//2 + 1):
        artists.append(member[i*2])
    detail = cells[3].text
    guests = ["戌亥とこ", "獅子神レオナ"]
    for guest in guests:
        if guest in detail:
            artists.append(guest)
    return artists

# あれ、カリとのライブは？？？

# video_idとt、アーティスト、曲名を取り出す
# 先頭だけ特殊なセルなためスキップする
tmp = []
for row in rows[1:]:
    cells = row.find_all('td')
    if check_cell(cells):
        videoid, time = get_videoid_time(cells)
        artists = get_artist(cells)
        artists_string = ','.join(artists)
        text = cells[2].text
        tmp.append([artists_string, text, videoid, time])
        
df = pd.DataFrame(data=np.array(tmp), 
                  columns=['Artists', 'SongTitle', 'VideoID', 'Time'])
df.to_csv('sample.csv', index=False)

In [ ]:
# オリ曲を管理しているテーブル
songs = soup.find_all("table", {"id":"content_block_26"})[0]
rows = songs.find_all("tr")

# メンバーに星街すいせいが含まれるかどうかでフィルタする必要がありそう
# あれ？ out side frameなくね？
# ここに関してはYoutubeへのリンクも無いし、数も少ないので手動入力でもいいかも

# セルごとの要素を抽出
# 公開日、メンバー、楽曲、リンク（Wikiへのリンク）

In [45]:
# 歌みた曲を管理しているテーブル
songs = soup.find_all("table", {"id":"content_block_34"})[0]
rows = songs.find_all("tr")

# セルごとの要素を抽出
# 公開日、メンバー、楽曲、備考
# メンバーに星街すいせいが含まれるかどうかでフィルタする必要がありそう
# ホロメン以外のメンバーがいる場合、備考の方に記載される
for row in rows[0:3]:
    for cell in row.find_all('td'):
        print(cell)

<td>2017/12/19</td>
<td>ときのそら</td>
<td><a class="outlink" href="https://youtu.be/RdmDf9WSMX0" rel="nofollow" target="_blank">ひいらぎ飾ろう</a></td>
<td>非公開</td>
<td>2017/12/22</td>
<td>ときのそら</td>
<td><a class="outlink" href="https://youtu.be/jtMvXgSVFjw" rel="nofollow" target="_blank">Winter Wonderland</a></td>
<td>非公開</td>
